In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **CNN2**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 60
NUM_CLASSES = 10

In [ ]:
# Data Augmentation and Normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
# Load and preprocess the dataset
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Music Genre Classification/images_original',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Music Genre Classification/images_original',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 800 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


In [ ]:
from keras.layers import Dropout

# Define the CNN model with Dropout regularization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),  # Adding Dropout regularization
   (6 Conv2D4, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),  # Adding Dropout regularization
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),  # Adding Dropout regularization
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Adding Dropout regularization
    Dense(NUM_CLASSES, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/60
25/25 [==============================] - 359s 14s/step - loss: 2.7230 - accuracy: 0.0950 - val_loss: 2.3021 - val_accuracy: 0.1094
Epoch 2/60
25/25 [==============================] - 55s 2s/step - loss: 2.3006 - accuracy: 0.1075 - val_loss: 2.2906 - val_accuracy: 0.1250
Epoch 3/60
25/25 [==============================] - 47s 2s/step - loss: 2.2812 - accuracy: 0.1088 - val_loss: 2.2744 - val_accuracy: 0.2135
Epoch 4/60
25/25 [==============================] - 45s 2s/step - loss: 2.2436 - accuracy: 0.1713 - val_loss: 2.2282 - val_accuracy: 0.1927
Epoch 5/60
25/25 [==============================] - 46s 2s/step - loss: 2.2228 - accuracy: 0.1850 - val_loss: 2.1919 - val_accuracy: 0.2292
Epoch 6/60
25/25 [==============================] - 46s 2s/step - loss: 2.1915 - accuracy: 0.2013 - val_loss: 2.1746 - val_accuracy: 0.2135
Epoch 7/60
25/25 [==============================] - 48s 2s/step - loss: 2.1836 - accuracy: 0.2087 - val_loss: 2.1223 - val_accuracy: 0.2448
Epoch 8/60
25/25 [

In [ ]:
# Save the trained model
model.save('CNN.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import librosa
from scipy import signal

In [ ]:
# Load the trained CNN model
model = load_model('CNN.h5')

# Define constants for preprocessing spectrogram images
IMAGE_SIZE = (128, 128)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load the trained CNN model
model = load_model('CNN.h5')

# Define constants for preprocessing images
IMAGE_SIZE = (128, 128)

# Prediction function
def predict_genre_from_image(image_file_path):
    try:
        # Step 1: Load the image and resize it to match the input size of the CNN model
        image = load_img(image_file_path, target_size=IMAGE_SIZE)
        # Convert image to array
        image_array = img_to_array(image)
        # Expand dimensions to create a batch
        image_batch = np.expand_dims(image_array, axis=0)

        # Step 2: Preprocess the image
        preprocessed_image = preprocess_input(image_batch)

        # Step 3: Use the trained model to predict the genre of the image
        prediction = model.predict(preprocessed_image)

        # Step 4: Decode the prediction and get the predicted genre
        predicted_class_index = np.argmax(prediction)
        genre_labels = [
            'Jazz', 'Blues', 'Country', 'Disco', 'HipHop',
            'Classical', 'Metal', 'Pop', 'Reggae', 'Rock'
        ]
        predicted_genre = genre_labels[predicted_class_index]

        return predicted_genre
    except Exception as e:
        print("Error occurred while predicting genre:", e)
        return None

# Example usage:
image_file_path = '/content/drive/MyDrive/Music Genre Classification/images_original/blues/blues00065.png'
predicted_genre = predict_genre_from_image(image_file_path)
print("Predicted genre:", predicted_genre)


1/1 [==============================] - 0s 99ms/step
Predicted genre: Classical
